In [33]:
# Import required libraries
import pandas as pd

# Read the CSV file
file_name = "../trades.csv"
df = pd.read_csv(file_name)

row_count = df.shape[0]
# drop header rows that appear multiple times in the csv file
df = df.drop(df[df.timestamp == "timestamp"].index)

restart_count = row_count - df.shape[0]

# timestamp,uuid,landed,accepted,rejected,errorType,errorContent,txn0Signature,txn1Signature,txn2Signature,arbSize,expectedProfit,hop1Dex,hop2Dex,sourceMint,intermediateMint,tipLamports,mempoolEnd,preSimEnd,simEnd,postSimEnd,calcArbEnd,buildBundleEnd,bundleSent

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['mempoolEnd'] = pd.to_datetime(df['mempoolEnd'], unit='ms')
df['preSimEnd'] = pd.to_datetime(df['preSimEnd'], unit='ms')
df['simEnd'] = pd.to_datetime(df['simEnd'], unit='ms')
df['postSimEnd'] = pd.to_datetime(df['postSimEnd'], unit='ms')
df['calcArbEnd'] = pd.to_datetime(df['calcArbEnd'], unit='ms')
df['buildBundleEnd'] = pd.to_datetime(df['buildBundleEnd'], unit='ms')
df['bundleSent'] = pd.to_datetime(df['bundleSent'], unit='ms')
df[['landed', 'rejected']] = df[['landed', 'rejected']].fillna(
    False).astype('bool')
df[['accepted', 'arbSize', 'expectedProfit', 'tipLamports']] = df[[
    'accepted', 'arbSize', 'expectedProfit', 'tipLamports']].astype('int64')

# Display the first few records
print(df.head())
print(df.info())

                timestamp                                               uuid  \
0 2024-01-23 18:03:40.928  219935f4c2d28e282025cc0295aad4cefe003d78afc0f9...   
1 2024-01-23 18:03:40.928  c15769919bcfd2dd2cdd5e7cf649be4cebece8130b547a...   
2 2024-01-23 18:03:40.928  af4f70b7063c10d99ae21e6ad9cbaa8c9a60ce51f35f01...   
3 2024-01-23 18:03:40.928  34b5670f1239864c5574943077447c27a27243a7002b98...   
4 2024-01-24 00:19:25.312  a51cde4e85fee38c88434a2925e10daae00425d637bd7c...   

   landed  accepted  rejected          errorType  \
0   False         0      True  simulationFailure   
1   False         0      True  simulationFailure   
2   False         0      True  simulationFailure   
3   False         0      True  simulationFailure   
4   False         0      True  simulationFailure   

                                        errorContent  \
0  {"txSignature":"4y2dvZqzdG9KEZCQMLPLt8E4H7rwcz...   
1  {"txSignature":"22wCw3SkkCpaMXN1nGBE4N4pArmp9d...   
2  {"txSignature":"5qYi65d8YecRdUeA2TS

/tmp/ipykernel_20512/2293968553.py:16: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
/tmp/ipykernel_20512/2293968553.py:17: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['mempoolEnd'] = pd.to_datetime(df['mempoolEnd'], unit='ms')
/tmp/ipykernel_20512/2293968553.py:18: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime stri

In [34]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable


In [35]:
print("Reading file: ", file_name)
print("First item timestamp (UTC): ", df['timestamp'].iloc[0])
print(f"Restart count: {restart_count}")

LAMPORTS_PER_SOL = 1000000000

# Calculate the percentage of rows where 'landed' is True compared to those with 'accepted' > 0
success_landed = df[(df['landed'] == True) & (df['accepted'] > 0)]
success_landed_count = success_landed.shape[0]
total_lamports_tipped = success_landed['tipLamports'].sum()
accepted_gt_0 = df[df['accepted'] > 0].shape[0]
if accepted_gt_0 > 0:
    percentage_landed_accepted = (success_landed_count / accepted_gt_0) * 100
else:
    percentage_landed_accepted = 0
# Calculate the percentage of rows with 'accepted' > 0 compared to the total
total_rows = df.shape[0]
percentage_accepted = (accepted_gt_0 / total_rows) * 100

# Print the results
print(f"Total sent: {total_rows}, accepted: {accepted_gt_0}, landed: {success_landed_count}")
print(
    f"Percentage landed of accepted: {percentage_landed_accepted:.2f}%")
print(
    f"Percentage accepted of sent: {percentage_accepted:.2f}%")
print(f"Total lamports tipped: {total_lamports_tipped} ({total_lamports_tipped / LAMPORTS_PER_SOL:.6f} SOL)")

# Find the error percentage for different error messages
errors_filtered = df[df['errorType'].notnull()]
sim_failures = errors_filtered[errors_filtered['errorType'].str.contains(
    "simulationFailure")]
total_errors = errors_filtered.shape[0]

sim_errors_my_fault = sim_failures[sim_failures.apply(
    lambda row: row['txn1Signature'] in row['errorContent'] or row['txn2Signature'] in row['errorContent'], axis=1)]
sim_errors_backrun_txn = sim_failures[sim_failures.apply(
    lambda row: row['txn0Signature'] in row['errorContent'], axis=1)]
sim_tx_already_processed = sim_failures[sim_failures['errorContent'].str.contains(
    "This transaction has already been processed")]
sim_other = sim_failures[~sim_failures.index.isin(
    sim_errors_my_fault.index.union(
        sim_errors_backrun_txn.index).union(sim_tx_already_processed.index))]

# Group non-simulation errors by errorType
non_sim_errors = errors_filtered[~errors_filtered['errorType'].str.contains("simulationFailure")]
sending_errors = errors_filtered[errors_filtered['errorType'].str.contains("sendingError")]

error_groups = non_sim_errors.groupby("errorType").size().reset_index(name='count')

# Calculate the error totals and percentages for sim errors and other errors
error_totals = {
    'sim_errors_my_fault': sim_errors_my_fault.shape[0],
    'sim_errors_backrun_txn': sim_errors_backrun_txn.shape[0],
    'sim_tx_already_processed': sim_tx_already_processed.shape[0],
    'sim_other': sim_other.shape[0],
}

error_percentages = {k: (v / total_errors) * 100 for k, v in error_totals.items()}
for _, row in error_groups.iterrows():
    error_name = f"{row['errorType']}"
    error_totals[error_name] = row['count']
    error_percentages[error_name] = (row['count'] / total_errors) * 100

# Remove errors with zero count
error_totals = {k: v for k, v in error_totals.items() if v > 0}
error_percentages = {k: v for k, v in error_percentages.items() if k in error_totals}

# Print the error totals and their percentages
print("\nError Totals and Percentages:")
for error_type, total in error_totals.items():
    percentage = error_percentages[error_type]
    print(f"{error_type}: Total: {total}, Percentage: {percentage:.2f}%")

Reading file:  ../trades.csv
First item timestamp (UTC):  2024-01-23 18:03:40.928000
Restart count: 9
Total sent: 183, accepted: 7, landed: 4
Percentage landed of accepted: 57.14%
Percentage accepted of sent: 3.83%
Total lamports tipped: 4552000 (0.004552 SOL)

Error Totals and Percentages:
sim_errors_my_fault: Total: 119, Percentage: 65.03%
sim_errors_backrun_txn: Total: 51, Percentage: 27.87%
sim_tx_already_processed: Total: 8, Percentage: 4.37%
sendingError: Total: 8, Percentage: 4.37%
stateAuctionBidRejected: Total: 1, Percentage: 0.55%


In [37]:
# 0x1771 is Jupiter SlippageToleranceExceeded
print(sim_errors_my_fault.tail(50).to_string())
print(sim_errors_backrun_txn.tail(50).to_string())
print(sending_errors.tail(50).to_string())

                  timestamp                                                              uuid  landed  accepted  rejected          errorType                                                                                                                                                                                                                                                                                                                                        errorContent                                                                             txn0Signature                                                                             txn1Signature                                                                             txn2Signature     arbSize  expectedProfit       hop1Dex       hop2Dex       hop3Dex                                    sourceMint                             intermediateMint1                             intermediateMint2  tipLamports              mempoolEnd     